## Load Dataset

In [4]:
import pandas as pd
import numpy as np
import janitor
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [5]:
import warnings

# Suppress warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [ ]:
# Define the path to 
patent_data = pd.read_csv(
    "C:\\Users\\shiri\\OneDrive\\Documents\\Python\\ml-projects\\data-606\\pharmaceutical-drug-price-prediction\\data\\raw\\patent.csv"
)

product_data = pd.read_csv(
    "C:\\Users\\shiri\\OneDrive\\Documents\\Python\\ml-projects\\data-606\\pharmaceutical-drug-price-prediction\\data\\raw\\products.csv"
)

exclusivity_data = pd.read_csv(
    "C:\\Users\\shiri\\OneDrive\\Documents\\Python\\ml-projects\\data-606\\pharmaceutical-drug-price-prediction\\data\\raw\\exclusivity.csv"
)

The dataset includes three main files:

1. **Products**: Contains details like ingredients, trade names, approval dates, and application types of pharmaceutical products.
2. **Patent**: Tracks patent numbers, expiration dates, and flags for drug substances or products.
3. **Exclusivity**: Provides information on exclusivity codes and their expiration dates for FDA-approved drugs.

These files together offer a comprehensive overview of pharmaceutical product approvals, patents, and exclusivity.

## Data Cleaning

In [8]:
# Clean column names in the same datasets
product_data = product_data.clean_names()
patent_data = patent_data.clean_names()
exclusivity_data = exclusivity_data.clean_names()

# Verify the cleaned column names
print("Cleaned product_data column names:")
print(product_data.columns)

print("\nCleaned patent_data column names:")
print(patent_data.columns)

print("\nCleaned exclusivity_data column names:")
print(exclusivity_data.columns)


Cleaned product_data column names:
Index(['ingredient', 'df;route', 'trade_name', 'applicant', 'strength',
       'appl_type', 'appl_no', 'product_no', 'te_code', 'approval_date', 'rld',
       'rs', 'type', 'applicant_full_name'],
      dtype='object')

Cleaned patent_data column names:
Index(['appl_type', 'appl_no', 'product_no', 'patent_no',
       'patent_expire_date_text', 'drug_substance_flag', 'drug_product_flag',
       'patent_use_code', 'delist_flag', 'submission_date'],
      dtype='object')

Cleaned exclusivity_data column names:
Index(['appl_type', 'appl_no', 'product_no', 'exclusivity_code',
       'exclusivity_date'],
      dtype='object')


In [9]:
# Function to clean and process the product_data
def clean_product_data(df):
    # 1. Clean column names using janitor
    df = df.clean_names()

    # 2. Separate 'df_route' into 'dosage_form' and 'route'
    if "df;route" in df.columns:
        df["dosage"] = df["df;route"].str.split(";").str[0]
        df["route"] = df["df;route"].str.split(";").str[-1]
        df.drop("df;route", axis=1, inplace=True)

    # 3. Make all categorical columns lowercase
    categorical_columns = df.select_dtypes(include="object").columns
    df[categorical_columns] = df[categorical_columns].apply(lambda x: x.str.lower())

    # 4. Convert 'approval_date' to date format
    if "approval_date" in df.columns:
        df["approval_date"] = pd.to_datetime(
            df["approval_date"], errors="coerce"
        ).dt.date

    return df

# Clean the dataset
product_data = clean_product_data(product_data)
# Display cleaned data
product_data.head()


,ingredient,trade_name,applicant,strength,appl_type,appl_no,product_no,te_code,approval_date,rld,rs,type,applicant_full_name,dosage,route
0,budesonide,budesonide,padagis israel,2mg/actuation,a,215328,1,ab,2023-04-12,no,yes,rx,padagis israel pharmaceuticals ltd,"aerosol, foam",rectal
1,budesonide,uceris,salix,2mg/actuation,n,205613,1,ab,2014-10-07,yes,no,rx,salix pharmaceuticals inc,"aerosol, foam",rectal
2,minocycline hydrochloride,amzeeq,journey,eq 4% base,n,212379,1,NaN,2019-10-18,yes,yes,rx,journey medical corp,"aerosol, foam",topical
3,azelaic acid,azelaic acid,teva pharms usa,15%,a,210928,1,NaN,2020-10-07,no,no,discn,teva pharmaceuticals usa inc,"aerosol, foam",topical
4,betamethasone valerate,betamethasone valerate,alembic,0.12%,a,215832,1,ab,2024-08-22,no,no,rx,alembic pharmaceuticals ltd,"aerosol, foam",topical


In [10]:
# Function to clean and process the patent_data
def clean_patent_data(df):
    # 1. Clean column names using janitor
    df = df.clean_names()

    # 2. Convert 'patent_expire_date_text' to date format
    if "patent_expire_date_text" in df.columns:
        df["patent_expire_date_text"] = pd.to_datetime(
            df["patent_expire_date_text"], errors="coerce"
        ).dt.date

        # Rename the column 'patent_expire_date_text' to 'patent_expire_date'
        df.rename(
            columns={"patent_expire_date_text": "patent_expire_date"}, inplace=True)

    if "submission_date" in df.columns:
        df["submission_date"] = pd.to_datetime(
            df["submission_date"], errors="coerce"
        ).dt.date

    return df

# Clean the dataset
patent_data = clean_patent_data(patent_data)
# Display cleaned data
patent_data.head()


,appl_type,appl_no,product_no,patent_no,patent_expire_date,drug_substance_flag,drug_product_flag,patent_use_code,delist_flag,submission_date
0,N,20610,1,7625884,2026-08-24,NaN,NaN,U-141,NaN,NaT
1,N,18613,1,7560445,2027-02-01,Y,Y,U-986,NaN,NaT
2,N,19734,2,8455524,2027-04-18,NaN,NaN,U-1029,NaN,2013-06-27
3,N,19734,5,8455524,2027-04-18,NaN,NaN,U-1029,NaN,NaT
4,N,20832,7,7422388,2027-04-25,NaN,Y,U-1397,NaN,NaT


In [11]:
# Function to clean and process the exclusivity_data
def clean_exclusivity_data(df):
    # 1. Clean column names using janitor
    df = df.clean_names()

    # 2. Convert 'exclusivity_date' to date format
    if "exclusivity_date" in df.columns:
        df["exclusivity_date"] = pd.to_datetime(
            df["exclusivity_date"], errors="coerce"
        ).dt.date

    return df


# Clean the dataset
exclusivity_data = clean_exclusivity_data(exclusivity_data)
# Display cleaned data
exclusivity_data.head()


,appl_type,appl_no,product_no,exclusivity_code,exclusivity_date
0,N,17031,1,RTO,2026-07-13
1,N,18680,1,D-193,2027-06-28
2,N,20263,9,NS,2026-04-14
3,N,20825,1,M-232,2025-01-28
4,N,20825,2,M-232,2025-01-28


In [12]:
# Datasets column names
print("product_data columns\n", product_data.columns)
print("\npatent_data columns\n", patent_data.columns)
print("\nexclusivity_data columns\n", exclusivity_data.columns)


product_data columns
 Index(['ingredient', 'trade_name', 'applicant', 'strength', 'appl_type',
       'appl_no', 'product_no', 'te_code', 'approval_date', 'rld', 'rs',
       'type', 'applicant_full_name', 'dosage', 'route'],
      dtype='object')

patent_data columns
 Index(['appl_type', 'appl_no', 'product_no', 'patent_no', 'patent_expire_date',
       'drug_substance_flag', 'drug_product_flag', 'patent_use_code',
       'delist_flag', 'submission_date'],
      dtype='object')

exclusivity_data columns
 Index(['appl_type', 'appl_no', 'product_no', 'exclusivity_code',
       'exclusivity_date'],
      dtype='object')


In [13]:
# Datasets shape
print("product_data shape\n", product_data.shape)
print("\npatent_data shape\n", patent_data.shape)
print("\nexclusivity_data shape\n", exclusivity_data.shape)


product_data shape
 (46207, 15)

patent_data shape
 (19305, 10)

exclusivity_data shape
 (2011, 5)


## Data Integration

In [14]:
# Perform a left join between product_data and patent_data on 'appl_no' and 'product_no'
merged_data = product_data.merge(patent_data, on=["appl_no", "product_no"], how="left")

# Display the merged dataset
merged_data.head()

,ingredient,trade_name,applicant,strength,appl_type_x,appl_no,product_no,te_code,approval_date,rld,...,dosage,route,appl_type_y,patent_no,patent_expire_date,drug_substance_flag,drug_product_flag,patent_use_code,delist_flag,submission_date
0,budesonide,budesonide,padagis israel,2mg/actuation,a,215328,1,ab,2023-04-12,no,...,"aerosol, foam",rectal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,budesonide,uceris,salix,2mg/actuation,n,205613,1,ab,2014-10-07,yes,...,"aerosol, foam",rectal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,minocycline hydrochloride,amzeeq,journey,eq 4% base,n,212379,1,NaN,2019-10-18,yes,...,"aerosol, foam",topical,N,10137200,2030-10-01,NaN,NaN,U-2647,NaN,2019-11-15
3,minocycline hydrochloride,amzeeq,journey,eq 4% base,n,212379,1,NaN,2019-10-18,yes,...,"aerosol, foam",topical,N,8865139,2030-10-01,NaN,Y,U-2647,NaN,2019-11-15
4,minocycline hydrochloride,amzeeq,journey,eq 4% base,n,212379,1,NaN,2019-10-18,yes,...,"aerosol, foam",topical,N,10398641,2037-09-08,NaN,NaN,U-2647,NaN,2019-11-15


In [15]:
# Perform a left join between merged_data and exclusivity_data on 'appl_no' and 'product_no'
final_data = merged_data.merge(
    exclusivity_data, on=["appl_no", "product_no"], how="left"
)

# Display the final merged dataset
final_data.head()

,ingredient,trade_name,applicant,strength,appl_type_x,appl_no,product_no,te_code,approval_date,rld,...,patent_no,patent_expire_date,drug_substance_flag,drug_product_flag,patent_use_code,delist_flag,submission_date,appl_type,exclusivity_code,exclusivity_date
0,budesonide,budesonide,padagis israel,2mg/actuation,a,215328,1,ab,2023-04-12,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,budesonide,uceris,salix,2mg/actuation,n,205613,1,ab,2014-10-07,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,minocycline hydrochloride,amzeeq,journey,eq 4% base,n,212379,1,NaN,2019-10-18,yes,...,10137200,2030-10-01,NaN,NaN,U-2647,NaN,2019-11-15,NaN,NaN,NaN
3,minocycline hydrochloride,amzeeq,journey,eq 4% base,n,212379,1,NaN,2019-10-18,yes,...,8865139,2030-10-01,NaN,Y,U-2647,NaN,2019-11-15,NaN,NaN,NaN
4,minocycline hydrochloride,amzeeq,journey,eq 4% base,n,212379,1,NaN,2019-10-18,yes,...,10398641,2037-09-08,NaN,NaN,U-2647,NaN,2019-11-15,NaN,NaN,NaN


In [16]:
final_data.columns

Index(['ingredient', 'trade_name', 'applicant', 'strength', 'appl_type_x',
       'appl_no', 'product_no', 'te_code', 'approval_date', 'rld', 'rs',
       'type', 'applicant_full_name', 'dosage', 'route', 'appl_type_y',
       'patent_no', 'patent_expire_date', 'drug_substance_flag',
       'drug_product_flag', 'patent_use_code', 'delist_flag',
       'submission_date', 'appl_type', 'exclusivity_code', 'exclusivity_date'],
      dtype='object')

## Final Dataset

In [17]:
# Keep only the necessary columns
patent_dataset = final_data[
    [
        "appl_no",
        "product_no",
        "ingredient",
        "trade_name",
        "strength",
        "dosage",
        "route",
        "te_code",
        "type",
        "approval_date",
        "patent_expire_date",
        "applicant",
        "applicant_full_name",
    ]
]

In [18]:
patent_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79317 entries, 0 to 79316
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   appl_no              79317 non-null  int64 
 1   product_no           79317 non-null  int64 
 2   ingredient           79317 non-null  object
 3   trade_name           79317 non-null  object
 4   strength             79252 non-null  object
 5   dosage               79317 non-null  object
 6   route                79317 non-null  object
 7   te_code              23971 non-null  object
 8   type                 79317 non-null  object
 9   approval_date        73375 non-null  object
 10  patent_expire_date   35503 non-null  object
 11  applicant            79317 non-null  object
 12  applicant_full_name  79317 non-null  object
dtypes: int64(2), object(11)
memory usage: 7.9+ MB


In [19]:
patent_dataset.shape

(79317, 13)

In [20]:
# Display the first few rows
patent_dataset.head(10)

,appl_no,product_no,ingredient,trade_name,strength,dosage,route,te_code,type,approval_date,patent_expire_date,applicant,applicant_full_name
0,215328,1,budesonide,budesonide,2mg/actuation,"aerosol, foam",rectal,ab,rx,2023-04-12,NaN,padagis israel,padagis israel pharmaceuticals ltd
1,205613,1,budesonide,uceris,2mg/actuation,"aerosol, foam",rectal,ab,rx,2014-10-07,NaN,salix,salix pharmaceuticals inc
2,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
3,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
4,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2037-09-08,journey,journey medical corp
5,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
6,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
7,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
8,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
9,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp


In [21]:
# Find and count full row duplicates
full_duplicates = patent_dataset[patent_dataset.duplicated()]
print(f"Full row duplicates: {len(full_duplicates)}")

# Display the first few duplicate rows
full_duplicates.head()

Full row duplicates: 27066


,appl_no,product_no,ingredient,trade_name,strength,dosage,route,te_code,type,approval_date,patent_expire_date,applicant,applicant_full_name
3,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
5,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
6,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
7,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp
8,212379,1,minocycline hydrochloride,amzeeq,eq 4% base,"aerosol, foam",topical,NaN,rx,2019-10-18,2030-10-01,journey,journey medical corp


In [22]:
print("Number of full duplicate rows:", patent_dataset.duplicated().sum())

Number of full duplicate rows: 27066


In [23]:
# Remove full duplicates and overwrite the dataset
patent_dataset = patent_dataset.drop_duplicates()

# Confirm removal
print("New shape of patent_dataset:", patent_dataset.shape)

New shape of patent_dataset: (52251, 13)


In [24]:
patent_dataset.nunique()

,0
appl_no,26202
product_no,53
ingredient,2677
trade_name,7407
strength,4323
dosage,118
route,80
te_code,24
type,3
approval_date,7861


In [25]:
patent_dataset.describe(include="all")

,appl_no,product_no,ingredient,trade_name,strength,dosage,route,te_code,type,approval_date,patent_expire_date,applicant,applicant_full_name
count,52251.000000,52251.000000,52251,52251,52186,52251,52251,21643,52251,46380,8473,52251,52251
unique,NaN,NaN,2677,7407,4323,118,80,24,3,7861,2350,1874,1936
top,NaN,NaN,aripiprazole,pregabalin,10mg,tablet,oral,ab,rx,2017-11-13,2025-06-24,watson labs,watson laboratories inc
freq,NaN,NaN,345,268,1679,21574,35542,14727,28543,165,27,1089,1221
mean,125841.811602,2.004842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,77267.833394,1.874226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,4.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,71018.500000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,88713.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,208085.500000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Define the path to save the CSV file
repo_path = r"C:\Users\shiri\OneDrive\Documents\Python\ml-projects\data-606\pharmaceutical-drug-price-prediction"
save_path = os.path.join(repo_path, "data", "interim")

# Ensure the directory exists
os.makedirs(save_path, exist_ok=True)

# Save the dataset as a CSV file
csv_file_path = os.path.join(save_path, "patent_dataset.csv")
patent_dataset.to_csv(csv_file_path, index=False)

print(f"Dataset saved successfully at: {csv_file_path}")

Dataset saved successfully at: /content/drive/My Drive/DATA_606/data/patent_dataset.csv
